## Exploring exceptions in F#
To install fsharp kernel in Jupyter, follow https://github.com/dotnet/interactive/blob/main/docs/NotebookswithJupyter.md

## Utility for introspection
Similar to Python's `dir()`
Borrowed from http://www.fssnip.net/hm/title/The-dir-function-of-Python

In [13]:
open System.Reflection

let printw width =
    let formatCode = sprintf "%%%ds" -width
    printf <| Printf.TextWriterFormat<string -> unit, unit>(formatCode)
    
let printMembers header column (ms:string list) =
    let printItems f =
        let hasValue = not << System.String.IsNullOrWhiteSpace
        if hasValue header then printfn "[%s]" header

        List.iteri f ms
        printf "\n\n"

    match ms with
    | [] -> ()
    | _ when ms.Length <= column -> printItems (fun _ -> printf "%s ")

    | otherwize ->
        let longest = ms |> List.maxBy (fun s -> s.Length)
        let width = longest.Length + 1
        printItems (fun i item ->
            if i % column = 0 && i <> 0 then printf "\n" ;
            printw width item)

let collectNames (ms:seq<#MemberInfo>) =
    ms
    |> Seq.map (fun m -> m.Name)
    |> Set.ofSeq
    |> Set.toList
    |> List.sort

let dirbase x =
    let t = x.GetType()
    t.GetMembers() |> collectNames

let dir x =
    let print header = collectNames >> printMembers header 3
    let t = x.GetType()
    t.GetMethods()    |> print "Methods"
    t.GetProperties() |> print "Properties"
    t.GetFields()     |> print "Fields"

## Simple function that returns an exception

In [14]:
let division (x: int) (y: int): Result<int, Exception> =
    try
        let result = x / y
        Ok result
    with 
    | e -> Error e

In [15]:
let result =  division 100 0

## Exception is displayed in notebook

In [16]:
result

ResultValue ErrorValue 0 { System.DivideByZeroException: Attempted to divide by zero.
 at FSI_0017.division(Int32 x, Int32 y): TargetSite: { Microsoft.FSharp.Core.FSharpResult`2[System.Int32,System.Exception] division(Int32, Int32): Name: division, DeclaringType: FSI_0017, ReflectedType: FSI_0017, MemberType: Method, MetadataToken: 100663366, Module: { RefEmit_InMemoryManifestModule: MDStreamVersion: 131072, FullyQualifiedName: <In Memory Module>, ModuleVersionId: 97eb2652-058e-45c4-bae5-291d0c5d6b4b, MetadataToken: 1, ScopeName: RefEmit_InMemoryManifestModule, Name: <In Memory Module>, Assembly: { FSI-ASSEMBLY, Version=0.0.0.0, Culture=neutral, PublicKeyToken=null: Location: { System.NotSupportedException: The invoked member is not supported in a dynamic assembly.
 at System.Reflection.Emit.InternalAssemblyBuilder.get_Location()
 at lambda_method210(Closure , InternalAssemblyBuilder )
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 57: TargetSite: { System.String get_Location(): Name: get_Location, DeclaringType: System.Reflection.Emit.InternalAssemblyBuilder, ReflectedType: System.Reflection.Emit.InternalAssemblyBuilder, MemberType: Method, MetadataToken: 100685432, Module: System.Private.CoreLib.dll, IsSecurityCritical: True, IsSecuritySafeCritical: False, IsSecurityTransparent: False, MethodHandle: System.RuntimeMethodHandle, Attributes: PrivateScope, Public, Virtual, HideBySig, SpecialName, CallingConvention: Standard, HasThis, ReturnType: System.String, ReturnTypeCustomAttributes: System.String, ReturnParameter: System.String, IsCollectible: False, IsGenericMethod: False, IsGenericMethodDefinition: False, ContainsGenericParameters: False, MethodImplementationFlags: Managed, .. }, Message: The invoked member is not supported in a dynamic assembly., Data: [ ], InnerException: <null>, HelpLink: <null>, Source: System.Private.CoreLib, HResult: -2146233067, StackTrace: at System.Reflection.Emit.InternalAssemblyBuilder.get_Location()
 at lambda_method210(Closure , InternalAssemblyBuilder )
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 57 }, CodeBase: { System.NotSupportedException: The invoked member is not supported in a dynamic assembly.
 at System.Reflection.Emit.InternalAssemblyBuilder.get_CodeBase()
 at lambda_method211(Closure , InternalAssemblyBuilder )
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}.cs:line 57: TargetSite: { System.String get_CodeBase(): Name: get_CodeBase, DeclaringType: System.Reflection.Emit.InternalAssemblyBuilder, ReflectedType: System.Reflection.Emit.InternalAssemblyBuilder, MemberType: Method, MetadataToken: 100685433, Module: System.Private.CoreLib.dll, IsSecurityCritical: True, IsSecuritySafeCritical: False, IsSecurityTransparent: False, MethodHandle: System.RuntimeMethodHandle, Attributes: PrivateScope, Public, Virtual, HideBySig, SpecialName, CallingConvention: Standard, HasThis, ReturnType: System.String, ReturnTypeCustomAttributes: System.String, ReturnParameter: System.String, IsCollectible: False, IsGenericMethod: False, IsGenericMethodDefinition: False, ContainsGenericParameters: False, MethodImplementationFlags: Managed, .. }, Message: The invoked member is not supported in a dynamic assembly., Data: [ ], InnerException: <null>, HelpLink: <null>, Source: System.Private.CoreLib, HResult: -2146233067, StackTrace: at System.Reflection.Emit.InternalAssemblyBuilder.get_CodeBase()
 at lambda_method211(Closure , InternalAssemblyBuilder )
 at Microsoft.DotNet.Interactive.Formatting.MemberAccessor`1.GetValueOrException(T instance) in D:\a\_work\1\s\src\Microsoft.DotNet.Interactive.Formatting\MemberAccessor{T}

## Error Value and get_ErrorValue are properties and methods of `result`

In [17]:
dir(result)

[Methods]
CompareTo       Equals          GetHashCode     
GetType         NewError        NewOk           
ToString        get_ErrorValue  get_IsError     
get_IsOk        get_ResultValue get_Tag         

[Properties]
ErrorValue  IsError     IsOk        
ResultValue Tag         



## But how do we access the ErrorValue?

In [18]:
result.ErrorValue


input.fsx (1,8)-(1,18) typecheck error The type 'Result<_,_>' does not define the field, constructor or member 'ErrorValue'. Maybe you want one of the following:
   Error



Cell not executed: compilation error

In [19]:
result.get_ErrorValue()


input.fsx (1,8)-(1,22) typecheck error The type 'Result<_,_>' does not define the field, constructor or member 'get_ErrorValue'.



Cell not executed: compilation error

In [2]:
Exception